## Drone Detection

In [3]:
# essential imports
import os
import csv
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

# import YOLO model from ultralytics package
from ultralytics import YOLO

# custom utilities
from utils import set_device

In [4]:
# set the device depending on available GPU
device = set_device()

Using device: mps:0


In [ ]:
# load pretrained YOLOv8 model
model = YOLO("yolov8m.pt")  # n for nano, s for small, m for medium, l for large, x for extra large

#### Try on drone video

In [ ]:
# source video path
data_path = './data/drone-tracking-datasets/'
video_path = os.path.join(data_path, 'dataset1/cam1.mp4')

# get sample frame and store width and height
cap = cv2.VideoCapture(video_path)
ret, sample_frame = cap.read()
cap.release()
sample_frame_rgb = cv2.cvtColor(sample_frame, cv2.COLOR_BGR2RGB)
frame_height, frame_width, _ = sample_frame.shape
print(f"Frame size: {frame_width} x {frame_height}")

# select tracker
tracker = "sort.yaml"  # or 'bytetrack.yaml', 'strongsort.yaml'

In [ ]:
# run detection on video
results = model.predict(
    source=video_path, 
    device=device,   # set the device
    tracker=tracker,
    #show=True, # display live output
    save=True,  # save output video to runs/detect/
    stream=True  # return a generator that yields results for each frame, doesn't store whole video in memory
)

In [ ]:
# test tracking

# save trajectory to CSV
csv_file = "trajectory_1.csv"

trajectory = []  # list to store (frame_id, x_center, y_center)

for frame_id, result in enumerate(results, start=1):
    if len(result.boxes) > 0:
        # Take first detection (assuming single drone)
        box = result.boxes[0].xywh[0]  # [x_center, y_center, width, height] in pixels
        x_center, y_center = box[0].item(), box[1].item()
    else:
        # If no detection, log NaN or skip
        x_center, y_center = float('nan'), float('nan')

    trajectory.append([frame_id, x_center, y_center])

# save to CSV
with open(csv_file, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["frame_id", "x_center", "y_center"])
    writer.writerows(trajectory)

print(f"Trajectory saved to {csv_file}")

In [ ]:
# Extract x, y values in pixels, skipping NaNs
x_vals = np.array([x for _, x, y in trajectory])
y_vals = np.array([y for _, x, y in trajectory])

# Threshold in pixels
distance_threshold = 50

# Prepare lists of segments
segments_x = []
segments_y = []

current_seg_x = []
current_seg_y = []

for i in range(len(x_vals)):
    x, y = x_vals[i], y_vals[i]
    
    if np.isnan(x) or np.isnan(y):
        # Missing detection → start new segment
        if current_seg_x:
            segments_x.append(current_seg_x)
            segments_y.append(current_seg_y)
            current_seg_x, current_seg_y = [], []
        continue
    
    if current_seg_x:
        # Compute distance to previous point
        dx = x - current_seg_x[-1]
        dy = y - current_seg_y[-1]
        dist = np.sqrt(dx**2 + dy**2)
        if dist > distance_threshold:
            # Jump too far → start new segment
            segments_x.append(current_seg_x)
            segments_y.append(current_seg_y)
            current_seg_x, current_seg_y = [], []

    # Add point to current segment
    current_seg_x.append(x)
    current_seg_y.append(y)

# Add last segment
if current_seg_x:
    segments_x.append(current_seg_x)
    segments_y.append(current_seg_y)

# Plot
plt.figure(figsize=(12, 8))
plt.imshow(sample_frame_rgb)
for seg_x, seg_y in zip(segments_x, segments_y):
    plt.plot(seg_x, seg_y, marker='o', color='red', linewidth=2, markersize=1)

#plt.title("Drone Trajectory Over Sample Frame (with breaks)")
plt.axis("off")
plt.show()


In [ ]:
# Prepare X and Y lists with NaNs for missing detections
x_vals = np.array([x if not np.isnan(x) else np.nan for _, x, y in trajectory])
y_vals = np.array([y if not np.isnan(y) else np.nan for _, x, y in trajectory])

plt.figure(figsize=(12, 8))
plt.imshow(sample_frame_rgb)

# Plot segments separately to break lines at NaNs
# Using np.ma (masked arrays) to ignore NaNs in line segments
plt.plot(np.ma.masked_invalid(x_vals), np.ma.masked_invalid(y_vals),
         marker='o', color='red', linewidth=2, markersize=2)

plt.title("Drone Trajectory Over Sample Frame")
plt.axis("off")
plt.show()

In [ ]:
# Filter out frames where drone was not detected
x_vals = [x for _, x, y in trajectory if not (x != x)]  # ignore NaNs
y_vals = [y for _, x, y in trajectory if not (y != y)]

# plot trajectory over sample frame
plt.figure(figsize=(12, 8))
plt.imshow(sample_frame_rgb)
plt.plot(x_vals, y_vals, marker='o', color='red', linewidth=2, markersize=4)
plt.title("Drone Trajectory Over Sample Frame")
plt.axis("off")
plt.show()

In [ ]:
# Counters
detected_count = 0
missed_count = 0

for result in results:
    if len(result.boxes) > 0:  # any detections in this frame?
        detected_count += 1
    else:
        missed_count += 1

print(f"Detected in {detected_count} frames")
print(f"Missed in {missed_count} frames")

In [ ]:
# for the detections in camera 0 in the first dataset
# there are 5334 frames total
# of these there are 2416 frames that have no detections
# so there are 2918 frames with detections
# 55% detection rate

# for the nano model
# Detected in 1606 frames
# Missed in 3728 frames
# 30% detection rate

# for the small model
# Detected in 2357 frames
# Missed in 2977 frames
# 44% detection rate, much better!

# for the medium model
# Detected in 2704 frames
# Missed in 2630 frames
# 50.7% detection rate, even better!


In [ ]:
count_zero = 0
total_frames = 0
file_name = os.path.join(data_path, 'dataset1/detections/cam0.txt')  

# Replace 'data.txt' with your file path
with open(file_name, "r") as f:
    for line in f:
        total_frames += 1
        # Skip empty lines
        if not line.strip():
            continue
        
        # Split line into columns
        cols = line.strip().split()
        
        # Convert 2nd and 3rd columns to float
        col2, col3 = float(cols[1]), float(cols[2])
        
        # Check if both are 0
        if col2 == 0.0 and col3 == 0.0:
            count_zero += 1

print(f"Number of rows where 2nd and 3rd columns are 0: {count_zero}")
print(f"Total number of frames: {total_frames}")

In [ ]:
# display results frame by frame inline, by iterating through the generator
# otherwise can open in external window with show=True above

for result in results:
    annotated_frame = result.plot()
    annotated_frame = cv2.cvtColor(annotated_frame, cv2.COLOR_BGR2RGB)

    plt.imshow(annotated_frame)
    plt.axis("off")
    clear_output(wait=True)  # clear previous cell output
    display(plt.gcf())

In [ ]:
# Grab first frame only
first_result = next(iter(results))

# Plot with bounding boxes
annotated_frame = first_result.plot()  # numpy array, BGR format
annotated_frame = cv2.cvtColor(annotated_frame, cv2.COLOR_BGR2RGB)

# Show with matplotlib
plt.figure(figsize=(10, 6))
plt.imshow(annotated_frame)
plt.axis("off")
plt.title("First Frame with YOLO Detections")
plt.show()

In [ ]:
# put training code into YOLO format

# <class_id> <x_center_norm> <y_center_norm> <width_norm> <height_norm>

# need to decide bbox size or compute as we go from the results?

In [ ]:
# Parameters
bbox_width = 20   # in pixels
bbox_height = 20  # in pixels
image_width = 1280  # replace with actual image width
image_height = 720  # replace with actual image height
class_id = 0

input_folder = "detections"  # folder with txt files
output_folder = "labels"     # YOLO format labels

os.makedirs(output_folder, exist_ok=True)

for file_name in os.listdir(input_folder):
    if not file_name.endswith(".txt"):
        continue

    with open(os.path.join(input_folder, file_name), "r") as f_in, \
         open(os.path.join(output_folder, file_name), "w") as f_out:

        for line in f_in:
            frame_id, x, y = line.strip().split()
            x = float(x)
            y = float(y)

            # Normalize
            x_norm = x / image_width
            y_norm = y / image_height
            w_norm = bbox_width / image_width
            h_norm = bbox_height / image_height

            f_out.write(f"{class_id} {x_norm:.6f} {y_norm:.6f} {w_norm:.6f} {h_norm:.6f}\n")
